# **런던자전거 수요 예측 (구조화)**

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!ls /content/drive/MyDrive/data/

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import missingno as msno
import pandas as pd
df = pd.read_csv("./drive/MyDrive/data/london_merged.csv", parse_dates=['timestamp'])
df.head()

In [ ]:
# 데이터의 타입과 구조
print('데이터의 구조는 : ', df.shape)
print('데이터의 타입은 : ', df.dtypes)
print('데이터의 컬럼은 : ', df.columns)

In [ ]:
# 데이터의 결측치 확인
df.isna().sum()

In [ ]:
# 결측치 시각화     
msno.matrix(df)
plt.show()

In [ ]:
df['year'] = df['timestamp'].dt.year
df['month'] = df['timestamp'].dt.month
df['dayofweek'] = df['timestamp'].dt.dayofweek
df['hour'] = df['timestamp'].dt.hour
df.head()

In [ ]:
# 데이터의 갯수 확인

# df['year'].value_counts()
# df['dayofweek'].value_counts()
df['weather_code'].value_counts()

In [ ]:
a, b = plt.subplots(1, 1, figsize = (10, 5))
sns.boxplot(df['year'], df['cnt'])

In [ ]:
a, b = plt.subplots(1, 1, figsize = (10, 5))
sns.boxplot(df['month'], df['cnt'])

In [ ]:
a, b = plt.subplots(1, 1, figsize = (10, 5))
sns.boxplot(df['dayofweek'], df['cnt'])

In [ ]:
a, b = plt.subplots(1, 1, figsize = (10, 5))
sns.boxplot(df['hour'], df['cnt'])

In [ ]:
 # 그래프 함수 만들기
 def plot_bar(data, feature):
     fig = plt.figure(figsize=(12, 3))
     sns.barplot(x = feature, y='cnt', data = data, palette = 'Set3', orient='v')

In [ ]:
plot_bar(df, 'hour')

In [ ]:
plot_bar(df, 'dayofweek')

# **아웃라이어 제거(시그마)**

3 시그마 규칙이란 어떤 상품의 데이터가 ±3σ 밖에 존재할 확률은 0.3%이기 때문에 이 범위를 벗어나는 상품은 불량으로 간주

In [ ]:
# 아웃라이어 제거 함수 만들기

def is_outliers(s):
    lower_limit = s.mean() - (s.std() * 3) # 하한값
    upper_limit = s.mean() + (s.std() * 3) # 상한값
    return ~s.between(lower_limit, upper_limit)

In [ ]:
df_out = df[~df.groupby('hour')['cnt'].apply(is_outliers)]

print('이상치 제거전 : ', df.shape)
print('이상치 제거후 : ', df_out.shape)

# **데이터 전처리**

In [ ]:
df_out.dtypes

In [ ]:
df_out['weather_code'] = df_out['weather_code'].astype('category')
df_out['season'] = df_out['season'].astype('category')
df_out['year'] = df_out['year'].astype('category')
df_out['month'] = df_out['month'].astype('category')
df_out['hour'] = df_out['hour'].astype('category')
df_out.dtypes

In [ ]:
# 컴퓨터가 활용하기 용이하도록 더미처리
df_out = pd.get_dummies(df_out, columns=['weather_code', 'season', 'year', 'month', 'hour'])

df_out.head

In [ ]:
df_out.shape

In [ ]:
df_y = df_out['cnt']
df_x = df_out.drop(['timestamp', 'cnt'], axis=1)
df_x

In [ ]:
print(df_y)
df_y.shape

# **훈련용, 테스트용 데이터분리**

sklearn.model_selection 라이브러리의 train_test_split을 사용하여 데이터를 분리

In [ ]:
# 훈련용, 테스트용 데이터 분리
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(df_x, df_y, random_state=66, test_size=0.3, shuffle=False)

In [ ]:
print('x_train의 구조는 : ', x_train.shape)
print('y_train의 구조는 : ', y_train.shape)
print('x_test의 구조는 : ', x_test.shape)
print('y_test의 구조는 : ', y_test.shape)

# **Keras를 이용한 딥러닝**

In [ ]:
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.callbacks import EarlyStopping


In [ ]:
model = Sequential()
model.add(Dense(units=160, activation='relu', input_dim=57))
model.add(Dense(units=60, activation='relu'))
model.add(Dense(units=20, activation='relu'))
model.add(Dense(units=1, activation='linear'))

In [ ]:
model.summary()

In [ ]:
# 모델 컴파일
model.compile(loss='mae', optimizer='adam', metrics=['mae'])
# 과적합 방지(내려가지않고 5번 올라가면 멈춤)
early_stopping = EarlyStopping(monitor='loss', patience=5, mode='min')
# 50번 반복, 훈련을할때 10%를 검증용으로 사용
history = model.fit(x_train, y_train, epochs=50, batch_size=1, validation_split=0.1, callbacks=[early_stopping])

In [ ]:
plt.plot(history.history['val_loss'])
plt.plot(history.history['loss'])
plt.title('loss')
plt.xlabel('Epochs')
plt.ylabel('loss')
plt.legend(['val_loss', 'loss'])
plt.show();

In [ ]:
# 예측 평가
y_predict = model.predict(x_test)
y_predict

# **랜덤 포레스트(Random forest)**

랜덤 포레스트는 훈련을 통해 구성해놓은 다수의 나무들로부터 분류 결과를 취합해서 결론을 얻는, 일종의 인기 투표(?) 같은 거다.<hr>

In [ ]:
# random forest 모형
# 수십, 수백개의 모형을 병렬로 한꺼번에 만들어서 평균을 낸다거나 투표를 내는 기법

In [ ]:
# rmse
from sklearn.metrics import mean_squared_error

def RMSE(y_test, y_predict):
    return np.sqrt(mean_squared_error(y_test, y_predict))

print('RMSE : ', RMSE(y_test, y_predict))

In [ ]:
from sklearn.ensemble import RandomForestRegressor
rf = RandomForestRegressor(n_estimators=100, random_state=16)
rf.fit(x_train, y_train)
rf_result = rf.predict(x_test)
print('RMSE : ', RMSE(y_test, rf_result))

# 딥러닝 보다 성능이 떨어짐

# **부스팅(Boosting)**
부스팅은 머신러닝 앙상블 기법 중 하나로 약한 학습기(weak learner)들을 순차적으로 여러개 결합하여 예측 혹은 분류 성능을 높이는 알고리즘<hr>

여러 개의 알고리즘이 순차적으로 학습-예측을 하면서 이전에 학습한 알고리즘의 예측이 틀린 데이터를 올바르게 예측할 수 있도록, 다음 알고리즘에, 가중치를 부여하여 학습과 예측을 진행하는 방식입니다.<br>

* 부스팅은 기본적으로 앙상블(Ensemble) 아이디어에서 Sequential이 추가된 형태입니다.
부스팅 알고리즘에는 다음과 같은 알고리즘이 있습니다.

* AdaBoost
* GBM(Gradient Boosting Machine)
* XGBoost
* LightBoost

In [ ]:
# boosting 모형
# 순차적으로, 앞서만든 모형을 참고하여 새로운 모형을 만듬

# XGB
from xgboost import XGBRegressor
xgb = XGBRegressor(n_estimators = 100, random_state=16, objective = 'reg:squarederror')
xgb.fit(x_train, y_train)
xgb_result = xgb.predict(x_test)

print('RMSE : ', RMSE(y_test, xgb_result))
## 랜덤 포레스트 보다 성능이 떨어짐

In [ ]:
# LightGBM
import lightgbm as lgb
from lightgbm import LGBMRegressor
lgb = LGBMRegressor(n_estimator=100, random_stat=26)
lgb.fit(x_train, y_train)
lgb_result = lgb.predict(x_test)

print('RMSE : ', RMSE(y_test, lgb_result))

# **모형별 비교**

딥러닝, 랜덤포레스트, XGB, LightGBM을 데이터프레임으로 합쳐서 그래프로 비교

In [ ]:
dnn = pd.DataFrame(y_predict) # 딥러닝 결과값
rf = pd.DataFrame(rf_result) # 랜덤포레스트 결과값
xgb = pd.DataFrame(xgb_result) # XGB 결과값
lgb = pd.DataFrame(lgb_result) # LightGBM 결과값

compare = pd.DataFrame(y_test).reset_index(drop=True)
compare.head()

In [ ]:
compare['dnn'] = dnn
compare['rf'] = rf
compare['xgb'] = xgb
compare['lgb'] = lgb
compare.head()

## 커널 밀도 그래프

In [ ]:
sns.kdeplot(compare['cnt'], shade=True, color='r')
sns.kdeplot(compare['dnn'], shade=True, color='b')
sns.kdeplot(compare['rf'], shade=True, color='y')
sns.kdeplot(compare['xgb'], shade=True, color='g')
sns.kdeplot(compare['lgb'], shade=True, color='k');